# Neural Network Models for Combined Classification and Regression

Algunos problemas de predicción requieren tanto valores numéricos como categóricos para la misma entrada de datos. Una manera de resolver estos problemas es utilizar modelos predictivos de regresión y clasificación, sobre los mismos datos, y utilizar los modelos secuencialmente.

Otra forma (y, a menudo, más efectiva), es desarrollar una sola red neuronal que pueda predecir tanto un valor numérico como un categórico para la misma entrada. Esto se conoce como un **multi-output model** y puede ser relativamente fácil de desarrollar con las librerías **Keras** y **TensorFlow**. 

## 1. Single Model for Regression and Classification

Un problema de tener dos modelos distintos (uno para regresión y otro para clasificación), es que las predicciones pueden diverger entre ellos. Con un solo modelo, se tiene la ventaja de que se puede actualizar y mantener más fácilmente, ofreciendo mayor consistencia en las predicciones de ambos tipos.

## 2. Separate Regresion and Classification Models

En esta sección, se elegirá un *dataset* real donde se necesiten predicciones de regresión y clasificación al mismo tiempo, y luego se desarrollarán modelos separados para cada tipo de predicción.


### 2.a Abalone Dataset

Este *dataset* describe los detalles físicos de *abalone* (abulón, tipo de molusco) y requiere predecir el número de anillos que posee, el cual es un aproximado de la edad de la creatura. La "edad" puede ser predicha como un valor numérico (en años) o una clase categórica (el año ordinal como una clase).

In [1]:
# load and summarize the abalone dataset
from pandas import read_csv
from matplotlib import pyplot
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/abalone.csv'
dataframe = read_csv(url, header=None)
# summarize shape
print(dataframe.shape)
# summarize first few lines
print(dataframe.head())

(4177, 9)
   0      1      2      3       4       5       6      7   8
0  M  0.455  0.365  0.095  0.5140  0.2245  0.1010  0.150  15
1  M  0.350  0.265  0.090  0.2255  0.0995  0.0485  0.070   7
2  F  0.530  0.420  0.135  0.6770  0.2565  0.1415  0.210   9
3  M  0.440  0.365  0.125  0.5160  0.2155  0.1140  0.155  10
4  I  0.330  0.255  0.080  0.2050  0.0895  0.0395  0.055   7


Se puede observar que existen 4177 ejemplos (filas) que se pueden utilizar para entrenar y evaluar el modelo y 9 *features* (columnas) incluyendo la variable de salida. Todas las variables de entrada, menos la primera, son numéricas. Para facilitar la preparación de datos, se eliminará la primera columna y se concentrará en modelar los valores numéricos de entrada.

### 2.b Regression Model

Además de separar las columnas de entrada y salida, también se forzarán todas las columnas de entrada a que sean de tipo *float* (el esperado por las redes neuronales) y guardar la cantidad de *input features*, que será necesaria para construir el modelo más adelante.

In [ ]:
...
# split into input (X) and output (y) variables
X, y = dataset[:, 1:-1], dataset[:, -1]
X, y = X.astype('float'), y.astype('float')
n_features = X.shape[1]

Seguidamente, se puede dividir el *dataset* en el conjunto de entrenamiento y el conjunto de evaluación. Vamos a utilizar una muestra aleatoria de 67% del *dataset* para entrenar el modelo y el 33% restante para evaluarlo.

In [ ]:
...
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

Seguidamente, se puede definir el modelo de un *multi layer perceptron*. Se tendrán dos capas ocultas, la primera con 20 nodos y la segunda con 10 nodos, ambas utilizando la función de activación ReLU y una inicialización de pesos *"he normal"* (una buena práctica). El número de capas y nodos fue elegido arbitrariamente. 

La capa de salida tendrá un solo nodo para predecir un valor numérico y una función de activación linear.

In [ ]:
...
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))

El modelo será entrenado para minimizar el MSE *loss function* utilizando la versión efectiva **Adam** del gradiente del descenso estocástico.

In [ ]:
...
# compile the keras model
model.compile(loss='mse', optimizer='adam')

El modelo se entrenará durante 150 *epochs* y tendrá un *mini-batch size* de 32 muestras, nuevamente, parámetros elegidos arbitrariamente.

In [ ]:
...
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=2)

Finalmente, una vez el modelo ha sido entrenado, se evaluará y se reportará el MSE.

In [ ]:
...
# evaluate on test set
yhat = model.predict(X_test)
error = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % error)

Al unir todas las partes, se tiene el siguiente código de una red neuronal MLP para el *dataset* tratado como un problema de regresión.

In [3]:
# regression mlp model for the abalone dataset
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/abalone.csv'
dataframe = read_csv(url, header=None)
dataset = dataframe.values
# split into input (X) and output (y) variables
X, y = dataset[:, 1:-1], dataset[:, -1]
X, y = X.astype('float'), y.astype('float')
n_features = X.shape[1]
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))
# compile the keras model
model.compile(loss='mse', optimizer='adam')
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=2)
# evaluate on test set
yhat = model.predict(X_test)
error = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % error)

Epoch 1/150
88/88 - 1s - loss: 59.4564
Epoch 2/150
88/88 - 0s - loss: 12.1797
Epoch 3/150
88/88 - 0s - loss: 8.0399
Epoch 4/150
88/88 - 0s - loss: 7.5015
Epoch 5/150
88/88 - 0s - loss: 7.2487
Epoch 6/150
88/88 - 0s - loss: 7.0616
Epoch 7/150
88/88 - 0s - loss: 6.9186
Epoch 8/150
88/88 - 0s - loss: 6.8042
Epoch 9/150
88/88 - 0s - loss: 6.6513
Epoch 10/150
88/88 - 0s - loss: 6.4886
Epoch 11/150
88/88 - 0s - loss: 6.2426
Epoch 12/150
88/88 - 0s - loss: 6.0605
Epoch 13/150
88/88 - 0s - loss: 5.8669
Epoch 14/150
88/88 - 0s - loss: 5.7058
Epoch 15/150
88/88 - 0s - loss: 5.5454
Epoch 16/150
88/88 - 0s - loss: 5.3999
Epoch 17/150
88/88 - 0s - loss: 5.2980
Epoch 18/150
88/88 - 0s - loss: 5.1967
Epoch 19/150
88/88 - 0s - loss: 5.1850
Epoch 20/150
88/88 - 0s - loss: 5.1084
Epoch 21/150
88/88 - 0s - loss: 5.0649
Epoch 22/150
88/88 - 0s - loss: 5.0192
Epoch 23/150
88/88 - 0s - loss: 4.9980
Epoch 24/150
88/88 - 0s - loss: 4.9707
Epoch 25/150
88/88 - 0s - loss: 4.9920
Epoch 26/150
88/88 - 0s - loss: 

Al correr el código se prepara el *dataset*, se entrena el modelo y se reporta un estimado de su error, en este caso, cercano a 1.5 (anillos).

Ahora, veamos un modelo similar para clasificación.

### 2.c Classification Model

## 3. Combined Regression and Classification Models